In [ ]:
using JuMP
using MadNLP
using ProgressMeter
using CairoMakie

In [ ]:
N = 10 # number of atoms (at least 4)
fps = 120 # frames per second = 1/\tau
pref_dist = 1/(N-1)
break_dist = 2*pref_dist
pot_min = -1
search_rad = 1.
diss_coeff = 0.
l2_dissipation = true
alpha = 10
two_sided_pull = true
animation_width = 800
hidpi_scaling = 4
file_name = "crack_evol"
fontsize = 16
only_video = true; # if false generate a folder of snapshots for each frame

In [ ]:
# Lennard-jones of squared distances instead of distances
function lennard_jones(cur_dist_sq; pref_dist=1, pot_min=-1)
    # the factor 2 assures that the global
    # minimum is attained at cur_dist == pref_dist
    q = pref_dist^2/(2*cur_dist_sq)
    -4*pot_min*(q^2 - q)
end
W(d) = lennard_jones(d, pref_dist=pref_dist, pot_min=pot_min)
xs = range(pref_dist^2/2.5, 5*pref_dist^2, length=1000)
lines(xs, W.(xs))

In [ ]:
function dirichlet_delta_pull(t; max_pull=1, time_pull=1)
    if t <= time_pull
        return max_pull*sin(pi/2*t/time_pull)
    else
        return max_pull
    end
end

function dirichlet_delta_pull_and_push(t; max_pull=1., time_total=1.)
    if t <= time_total
        return max_pull*sin(pi*t/time_total)
    else
        return 0
    end
end

max_pull = 2*pref_dist
time_total = 3.
time_horizon = time_total
dirichlet_delta(t) = dirichlet_delta_pull_and_push(t, max_pull=max_pull, time_total=time_total)

g_right(t) = 1 + dirichlet_delta(t)
if two_sided_pull
    g_left(t) = -dirichlet_delta(t)
else
    g_left(t) = 0
end

xs = range(0, time_horizon, length=1000)
fig = Figure()
ax = Axis(fig[1,1])
lines!(ax, xs, g_right.(xs), color=:blue)
lines!(ax, xs, g_left.(xs), color=:orange)
fig

In [ ]:
soft_max(x, y) = (x*exp(alpha*x) + y*exp(alpha*y)) / (exp(alpha*x) + exp(alpha*y))
xs = range(-5*pref_dist^2, 5*pref_dist^2, length=1000)
soft_ys = [soft_max(x, 0.) for x in xs]
ys = [max(x, 0.) for x in xs]
empty!(ax)
lines!(ax, xs, soft_ys, color=:blue)
lines!(ax, xs, ys, color=:orange)
fig

In [ ]:
function initialize_jump_on_m1()
    N = 3
    step = 2
    simple_opt = Model(
        optimizer_with_attributes(MadNLP.Optimizer, "print_level" => MadNLP.WARN)
    )

    @NLparameter(simple_opt, prev_y[i=1:N] == (i-1)/(N-1))

    @variable(
        simple_opt,
        value(prev_y[i]) - search_rad <= y[i=1:N] <= value(prev_y[i]) + search_rad,
        start=value(prev_y[i])
    )


    @constraint(simple_opt, dirichlet_right, y[N] == g_right((step-1)/fps))
    @constraint(simple_opt, dirichlet_left, y[1] == g_left((step-1)/fps))

    @expression(simple_opt, dist_sq[i=1:N-1], (y[i+1] - y[i])^2)
    @NLexpression(
        simple_opt, energy, 
        1/(N-1)*sum(dist_sq[i] for i in 1:N-1)
    )

    @NLexpression(
        simple_opt, dissipation,
        0.5/(N-2)*sum((y[i] - prev_y[i])^2 for i in 2:N-1)
    )
    @NLobjective(simple_opt, Min, energy + diss_coeff*fps*dissipation)
    
    optimize!(simple_opt)
end;

# initialize_jump_on_m1()

In [ ]:
step = 2
minmove = Model(
    optimizer_with_attributes(
        MadNLP.Optimizer,
        "print_level" => MadNLP.WARN,
        "blas_num_threads" => 4
    )
)

@NLparameter(minmove, new_left == g_left((step-1)/fps))
@NLparameter(minmove, new_right == g_right((step-1)/fps))
@NLparameter(minmove, prev_y[i=1:N-2] == i*pref_dist)
@NLparameter(minmove, max_elong_sq[i=1:N-1] == pref_dist^2)
# @NLparameter(minmove, broken[i=1:N-1] == 0.) # 1 if broken 0 if not

@variable(
    minmove,
    -max_pull*two_sided_pull <= y[i=1:N-2] <= 1 + max_pull,
)

i0 = (N+1)÷2-1
# 4 -> 2-3 -> i0 == 1
# 5 -> 3-4 -> i0 == 2
# ...
for i in 1:i0
    set_start_value(y[i], value(new_left) + i*pref_dist)
end
for i in i0+1:N-2
    set_start_value(y[i], i*pref_dist + value(new_right) - (N-1)*pref_dist)
end

# Regularization
@constraint(minmove, pos_det[i=1:N-3], y[i+1] - y[i] >= pref_dist/10)

# energy
@expression(minmove, dist_sq[i=1:N-3], (y[i+1] - y[i])^2)
# @NLexpression(
#     minmove, damaged_dist_sq[i=1:N-3],
#     (
#         dist_sq[i]*exp(alpha*dist_sq[i]) 
#         + max_elong_sq[i]*exp(alpha*max_elong_sq[i])
#     ) / (exp(alpha*dist_sq[i]) + exp(alpha*max_elong_sq[i]))
# )
register(minmove, :W, 1, W, autodiff = true)
# @NLexpression(
#     minmove, energy, 
#     1/(N-1)*sum(W(broken[i]*damaged_dist_sq[i] + (1-broken[i])*dist_sq[i]) for i in 1:N-3)
# )
@NLexpression(
    minmove, energy, 
    W((y[1] - new_left)^2) + sum(W(dist_sq[i]) for i in 1:N-3) + W((new_right - y[N-2])^2)
)

# dissipation
if l2_dissipation
    @NLexpression(minmove, dissipation, 0.5*sum((y[i] - prev_y[i])^2 for i in 1:N-2))
else
    # implement Kelvin-Voigt
end

# objective
@NLobjective(minmove, Min, energy + diss_coeff*fps*dissipation)

minmove

In [ ]:
optimize!(minmove)

In [ ]:
minmoves = [[0., value.(prev_y)..., 1.], [value(new_left), value.(y)..., value(new_right)]];

In [ ]:
@showprogress "Computing minmoves..." for step in 3:fps*time_horizon
#     # update maximal elongation
#     yval = value.(y)
#     comp_max_elong_sq = (yval[2:end] - yval[1:end-1]).^2
#     new_max_elong_sq = max.(value.(max_elong_sq), comp_max_elong_sq)
#     set_value.(max_elong_sq, new_max_elong_sq)
    
#     # check for broken links
#     new_broken = [max_dist > break_dist ? 1. : 0. for max_dist in new_max_elong_sq]
#     set_value.(broken, new_broken)
    
    set_value.(prev_y, value.(y))
    
    # update boundary conditions
    set_value(new_left, g_left((step-1)/fps))
    set_value(new_right, g_right((step-1)/fps))
    
    optimize!(minmove)
    
    push!(minmoves, [value(new_left), value.(y)..., value(new_right)])
end

In [ ]:
function animate_steps(
        minmoves, file_name; 
        fps=30, width=600, fontsize=16, scale=4, save_snapshots=false
)
    # create necessary folders
    experiments_folder = "experiments"
    snapshots_folder = "$experiments_folder/$file_name"
    if !ispath(experiments_folder)
        mkdir(experiments_folder)
    end
    if save_snapshots
        if !ispath(snapshots_folder)
            mkdir(snapshots_folder)
        end
    end
    
    aspect = 4
    ax_aspect = 4
    height = width/aspect
    max_right = maximum([y[end] for y in minmoves])
    min_left = minimum([y[1] for y in minmoves])
    delta = 0.1
    max_len = max_right - min_left + 2delta
    ax_height = max_len/ax_aspect
    N = length(minmoves[1])
    
    fig = Figure(resolution=(scale*width, scale*height), fontsize=scale*fontsize)
    ax = Axis(
        fig[1, 1],
        limits=(min_left-delta, max_right+delta, -ax_height/2, ax_height/2),
        aspect=ax_aspect,
    )
    
    y = zeros(N)
    
    frame = 1
    record(fig, "experiments/$file_name.mp4", minmoves; framerate = fps) do (x)
        empty!(ax)
        scatter!(x[2:end-1], y[2:end-1], color=:blue, markersize=scale*6)
        scatter!(x[[1,end]], y[[1,end]], color=:orange, markersize=scale*6)
        if save_snapshots
            save("$snapshots_folder/$(file_name)_$frame.png", fig)
        end
        frame += 1
    end
end

In [ ]:
animate_steps(
    minmoves, file_name, 
    fps=60, 
    width=animation_width, 
    fontsize=fontsize,
    scale=hidpi_scaling,
    save_snapshots=!only_video
)